## PREPROCESSING DATA

In [1]:
from preprocessing import Parser, ReportPreprocessor, SrcPreprocessor
from datasets import DATASET

from collections import defaultdict

In [2]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nhatm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\nhatm\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [3]:
parser = Parser(DATASET)
bug_reports = parser.report_parser()
src_files = parser.src_parser()

In [4]:
src_processor = SrcPreprocessor(src_files, parser.name).preprocess()
report_processor = ReportPreprocessor(bug_reports, parser.name).preprocess()

## INDEXING SOURCE CODE FILES


In [3]:
import pandas as pd
import ast

In [6]:
file_path = "outputs/tomcat_source_code_data.csv"
src_df = pd.read_csv(file_path)

In [4]:
def build_inverted_index(documents):
    inverted_index = defaultdict(set)
    for key, contents in documents:
        for content in contents:
            inverted_index[content].add(key)
    return inverted_index

In [7]:
src_df['all_content'] = src_df['all_content'].apply(ast.literal_eval)
src_df['all_content'] = src_df['all_content'].apply(lambda x: x['stemmed'])

In [8]:
data_src_codes_content = src_df[['key', 'all_content']].values
data_src_codes_size = src_df[['key', 'total_lines']].values.tolist()

In [9]:
inverted_index_source_codes = build_inverted_index(data_src_codes_content)

In [10]:
len(inverted_index_source_codes)

23097

## BUILD VSM

In [11]:
import math

In [12]:
def compute_ltc(documents, inverted_index):
    tf_idf = defaultdict(lambda: defaultdict(float))
    total_documents = len(documents)

    for key, contents in documents:
        content_frequency = defaultdict(int)
        for content in contents:
            content_frequency[content] += 1

        for content, count in content_frequency.items():
            tf = 1 + math.log(count, 10)
            df = len(inverted_index.get(content, []))
            idf = math.log(total_documents / df, 10)

            tf_idf[key][content] = tf * idf

    for doc_id in tf_idf:
        norm = math.sqrt(sum(weight ** 2 for weight in tf_idf[doc_id].values()))
        if norm > 0:
            for content in tf_idf[doc_id]:
                tf_idf[doc_id][content] /= norm

    return tf_idf

In [13]:
vsm = compute_ltc(data_src_codes_content, inverted_index_source_codes)

In [14]:
len(data_src_codes_content)

1794

## CALCULATE COEFFICIENT FILE SIZE

In [15]:
max_size = 0
min_size = math.inf
for data_size in data_src_codes_size:
    if data_size[1] > max_size:
        max_size = data_size[1]
    if data_size[1] < min_size:
        min_size = data_size[1]

In [16]:
def calculate_coefficient_size(size):
    value_standardization = (size - min_size) / (max_size - min_size)
    return 1/(1 + math.exp(-value_standardization))

In [17]:
coefficient_size = defaultdict(float)
for data_size in data_src_codes_size:
    coefficient_size[data_size[0]] = calculate_coefficient_size(data_size[1])

## HANDLE REPORT DATA

In [18]:
file_path = "outputs/tomcat_bug_reports.csv"
report_df = pd.read_csv(file_path)

In [19]:
report_df.pos_tagged_description = report_df.pos_tagged_description.apply(ast.literal_eval)
report_df.pos_tagged_description = report_df.pos_tagged_description.apply(lambda x: x['stemmed'])

report_df.pos_tagged_summary = report_df.pos_tagged_summary.apply(ast.literal_eval)
report_df.pos_tagged_summary = report_df.pos_tagged_summary.apply(lambda x: x['stemmed'])

report_df['combined'] = report_df.apply(lambda row: row['pos_tagged_summary'] + row['pos_tagged_description'] if isinstance(row['pos_tagged_summary'], list) and isinstance(row['pos_tagged_description'], list) else [], axis=1)

In [20]:
report_data = report_df[['key', 'combined']].values.tolist()

In [21]:
report_data[0]

[56012, ['bug', 'statement', 'gener', 'use', 'attribut', 'page', 'direct']]

In [22]:
def compute_lnc(query):
    tf_idf = defaultdict(float)

    term_freq = defaultdict(int)
    for term in query[1]:
        term_freq[term] += 1

    for term, freq in term_freq.items():
        tf = 1 + math.log10(freq)
        tf_idf[term] = tf * 1

    norm = math.sqrt(sum(weight ** 2 for weight in tf_idf.values()))
    if norm > 0:
        for term in tf_idf:
            tf_idf[term] /= norm

    return tf_idf


In [23]:
len(coefficient_size)

1794

## CALCULATE rVSM

In [27]:
def calculate_score(query, documents, inverted_index):
    query_vec = compute_lnc(query)
    doc_vecs = compute_ltc(documents, inverted_index)

    scores = {}

    for key, doc_vec in doc_vecs.items():
        dot_product = 0.0
        for term in query_vec:
            dot_product += query_vec[term] * doc_vec.get(term, 0.0)

        dot_product *= coefficient_size[key]

        scores[key] = dot_product

    return scores

## GET PREVIOUS BUG REPORT

In [42]:
report_df['report_time'] = pd.to_datetime(report_df['report_time'])
df_sorted_by_key = report_df.sort_values(by='key')
df_filtered_time = df_sorted_by_key[['key', 'combined', 'report_time', 'fixed_files']]

In [43]:
df_filtered_time.head(5)

,key,combined,report_time,fixed_files
545,10526,"[bug, authent, cach, princip, user, authent, p...",2002-07-06 13:11:40,java\org\apache\catalina\authenticator\Authent...
544,10972,"[bug, realm, classnam, produc, nullpointerexce...",2002-07-19 02:30:07,java\org\apache\catalina\startup\Catalina.java...
955,11117,"[bug, coyot, connector, deal, put, use, chunk,...",2002-07-24 07:44:57,java\org\apache\coyote\http11\filters\ChunkedI...
543,13731,"[bug, request, respons, session, variabl, prov...",2002-10-17 13:23:42,java\org\apache\jasper\compiler\Generator.java .
526,15688,"[bug, name, import, hi, jasper, gener, compil,...",2002-12-27 22:30:47,java\org\apache\jasper\compiler\Generator.java...


In [31]:
previous_reports_data = df_filtered_time.values.tolist()

In [33]:
def build_inverted_index_for_report(documents):
    inverted_index = defaultdict(set)
    for key, contents, files in documents:
        for content in contents:
            inverted_index[content].add(key)
    return inverted_index

In [34]:
inverted_index_previous_report_data = build_inverted_index_for_report(previous_reports_data)

In [ ]:
def get_previous_report_data(current_report):
    previous_report_data = inverted_index_previous_report_data